# Introduction :
Ceci est un Notebook qui explique l'utilisation du service d'optimisation du moteur. 

**Principe de fonctionnement** : 

Le programme fonctionne en croisant des données statiques et dynamiques pour résoudre un problème d'optimisation sur une fenêtre temporelle donnée :
* **Inputs** : Un objet `Client` (paramètres physiques, contraintes) + un `DataFrame` de production solaire (prévisions).
* **Processus** : Le service projette ces données sur un **horizon** défini (ex: 24h) et appelle un solveur pour trouver la séquence de chauffe optimale.
* **Output** : Une trajectoire de commande (Températures, Puissances, Coûts).

**Cas d'utilisation : L'Horizon Glissant** : 

Ce service est conçu pour fonctionner en boucle fermée (principe de l'horizon glissant) :
1.  À l'instant *t*, on calcule l'optimum sur *t + Horizon*.
2.  On applique la décision immédiate.
3.  À *t + pas*, on relance le calcul avec les données mises à jour (nouvelle température initiale, météo actualisée).

**Architecture technique** : 

L'utilisation du moteur repose sur 3 briques logicielles distinctes :
1.  **`Client`** : Configuration des données métier (Voir notebook *client_creation*).
2.  **`OptimizerService`** : Orchestrateur technique qui exécute les calculs.
3.  **`TrajectorySystem`** : Objet de données contenant les résultats (vecteurs numpy).

L'utilisateur de ce moteur doit savoir gérer ces 3 briques. 

**Objectif du Notebook**
Ce document se concentre uniquement sur la brique **`OptimizerService`**.
Nous allons détailler son instanciation, sa configuration (horizon, pas de temps) et l'appel de ses méthodes de calcul (Optimisation vs Simulation Standard vs Routeur).

# Les imports : 
Le service est implémenté via la classe *OptimizerService*, celle-ci peut être importée directement via : 

In [1]:
from optimiser_engine import OptimizerService 

# Préparation du Contexte (Inputs)
 
Pour que le service fonctionne, il lui faut deux éléments essentiels en plus du `Client` :
1.  Une **Date de départ** (`start_datetime`).
2.  Des **Données Solaires** (`production_df`) couvrant l'horizon de temps.
3. Une **température initiale** (`initial_temp`) 


## Client : 
La classe **Client** a été détaillé dans le Notebook "client_creation.ipynb". Ici, pour aller vite, on crée un client à partir du fichier YAML (voir fin du notebook) "client_sample.yaml" qui se trouve dans le même dossier que ce Notebook. 

Pour faire cela, on n'a besoin que de la classe *Client*, les classes imbriqués, ne sont pas nécessaires pour cette création simplifiée. (Voir plus de détail dans le Notebook client_creation.ipynb). 

In [2]:
# On importe 
from optimiser_engine import Client 

Voici le programme qui extrait le client à partir du fichier. 

In [3]:
# Création rapide d'un client à partir de client_sample.yaml qui existe dans le même dossier que ce IPYNB.

from pathlib import Path 

BASE_DIR = Path.cwd()
file = BASE_DIR / "client_sample.yaml" 

client = Client.from_yaml_file(file)  
print(client)

client : 12345


## Tableau des prévisions solaires : 

En plus, du client, le service exige un DataFrame Pandas de deux colonnes, une pour les dates et une pour la production prévue (en Watts). 

Le tableau doit respecter ces conditions : 

- Le tableau doit contenir la colonne des dates (sinon on lève une erreur). 
- Les dates ne doivent pas être trop espacées. 
- La colonne des dates doit couvrir l'intervalle [instant_initial ; instant_initial + horizon] (voir section suivante). En effet, si cette colonne ne couvre pas l'intervalle, cela veut dire qu'on cherche une optimisation pour des dates où on connaît pas la météo, ce qui n'est pas possible. 

Le fichier `weather.csv` contient une structure d'un DataFrame qui respecte ces exigences pour les deux journées (01/01 et 02/01). 

On va créer un Pandas.Df à partir de ce CSV : 

- On importe Pandas : 

In [4]:
import pandas as pd

- On accède au chemin vers le fichier CSV :

In [5]:
# BASE_DIR est définie lorsqu'on a cherché le client - C'est le chemin vers le dossier parent de ce Notebook. 
path_CSV = BASE_DIR / "weather.csv" 

- On appelle la méthode `read_csv`

In [6]:
df_productions = pd.read_csv(path_CSV, parse_dates=["date"]).set_index("date")



## Instant initial : 
Il faut simplement l'instant (type `datetime`) qui correspond à l'instant initial qu'on regarde. 
Comme dit avant, il faut que l'instant initial et l'horizon (expliqué dans la partie instanciation) soient couvert par le DF de la production. 

Voici un exemple de création d'un instant initial :

- On importe `datetime` 

In [7]:
from datetime import datetime 

- On crée la date 01/01 à 2h du matin, on choisira ensuite l'horizon (24h), ce qui sera parfaitement couvert. 

In [8]:
instant_initial = datetime(2026, 1, 1, 2, 0) 

## Température initiale : 
Il s'agit de la température durant l'instant initial. Cette température doit être un nombre (float ou bien int). 
Cette température initiale ne doit pas dépasser la valeur `OptimizerService.MAX_WATER_HEATER_TEMP` et ne doit pas être en dessous de `OptimizerService.MIN_WATER_HEATER_TEMP`. (Voir partie suivate sur l'instanciation).


Voici un exemple de création de la température initiale (50°C). 

In [9]:
temperature_initiale = 50 

# Instanciation du service :
Avant de pouvoir calculer les trajectoires, il faut d'abord créer (en instanciant la classe *OptimizerService*) un service. 
Un service s'instancie avec la valeur souhaitée de l'horizon et la valeur souhaitée du pas de temps. Si ces valeurs ne sont pas fournies, les valeurs par défauts de 24h (pour l'horizon) et 15min (pour le pas) seront prises. 


### Création d'un service par défaut :
Voici l'exemple de code pour créer un service (horizon standard de 24h et Pas de temps de 15 min) : 

In [10]:
service = OptimizerService() 

### Création d'un service personnalisée : 
Pour personnaliser l'horizon et le pas de temps, il faut mettre ces paramètres lors de l'initialisation, voici un exemple qui permet de faire cela pour un horizon personnalisé de 30h et un pas de 10 min :


In [11]:
horizon_pers = 30
pas_pers = 10 
service_pers = OptimizerService(horizon_hours=horizon_pers, step_minutes=pas_pers) 

NOTE : Dans cette création personnalisée, il faut respecter des règles : 
- L'horizon ne doit pas dépasser `OptimizerService.MAX_HOURS_HORIZON` et ne doit pas être en dessous de `OptimizerService.MIN_HOURS_HORIZON`. 
- Le pas de temps ne dois pas être en dessous de `OptimizerService.min_step_minutes` et ne peut pas dépasser la moitié de l'horizon. 

Si l'une de ces règles n'est pas respectée, la création du service ne peut pas aboutir. 

### Accès aux attributs : 
Pour accèder aux attributs d'un service, on procède de la même manière que pour toute classe, voici un exemple qui applique ceci pour les deux services créés précédemment : 

In [12]:
# Pour accéder aux horizons : 
horizon1 = service.horizon 
horizon_perso = service_pers.horizon 
print(horizon1, horizon_perso)  
# Pour accéder aux pas de temps : 
pas1 = service.step_minutes 
pas_perso = service_pers.step_minutes 
print(pas1, pas_perso)
# Pour accéder aux attributs de la classe : 
min_step = OptimizerService.min_step_minutes 
max_horizon = OptimizerService.MAX_HORIZON_HOURS
min_horizon = OptimizerService.MIN_HORIZON_HOURS 
print(min_step, min_horizon, max_horizon)


24 30
15 10
5 1 48


### IMPORTANT : La température initiale maximale :
Il est possible, pour des raisons de sécurité, d'imposer une limite maximale et une limite minimale à la température initiale que prendra le solveur. 
Cela se fait via l'accès aux attributs de la classe OptimizerService, voici un exemple :

In [13]:
OptimizerService.MAX_WATER_HEATER_TEMP = 95 
OptimizerService.MIN_WATER_HEATER_TEMP = 10 

# Génération de la trajectoire optimisée :
Une fois qu'on a crée un service, il est possible d'appeler le service pour générer la trajectoire optimisée pour la partie [instant_initial, instant_initial + horizon]. 

Cela s'effectue via la fonction *trajectory_of_client*, en fournissant le client, le DF des prévisions solaires, l'instant initial et la température initiale. 

Cette fonction génère ce qu'on appelle une trajectoire, c'est un objet d'une classe **TrajectorySystem**, la manipulation de trajectoires est détaillé dans le Notebook "trajectory_manipulation.ipynb". 

La trajectoire générée par cette fonction est une trajectoire optimisée, c'est à dire la meilleure compte tenu des paramètres solaires, des paramètres techniques, les exigences, les contraintes et la température initiale. 

Voici un exemple d'utilisation pour les éléments créés précedemment : 

In [14]:
trajectory_optimized = service.trajectory_of_client(client = client, 
                                                    start_datetime=instant_initial, 
                                                    initial_temperature=temperature_initiale, 
                                                    production_df=df_productions) 


/Users/anaselb/Dev/libs_optimasol/optimiser_engine_v2.0/src/optimiser_engine/engine/service.py:384: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  target_index = pd.date_range(


NOTE : 
- Si la règle de compatibilité entre start_datetime, l'horizon et production_df n'est pas respectée, cette fonction ne peut pas aboutir. 
- Si le solveur échoue, on lève une erreur *SolverFailed* et on rapport le message du solveur. Généralement ce genre d'errers provient uniquement lorsque le problème est non faisable (unfeasible). Dans le cas de l'optimisation, lorsque le problème est unfeasible, il faut chauffer à 100%. 

# Génération de la trajectoire standard :
Afin de pouvoir comparer le scénario optimisé avec le scénario standard où il y a uniquement un chauffe-eau sans routeur, ni logiciel d'optimisation, le service est doté d'une méthode **trajectory_of_client_standard** qui permet de générer cette trajectoire standard qui imite le scénario standard. 

On distingue entre deux cas standards : 
- Les chauffes eaux résistifs classiques qui s'allument quand on descend en dessous de la température de consigne et qui s'éteignent dès qu'on est en dessus. 
- Les chauffes eaux qui appliquent cette règle, mais ne s'allument que pendant les heures creuses HC. 

Cette distinction est gérée par une classe Enum **StandardWHType**. Elle prend deux valeurs *SETPOINT* (pour le cas 1) et *SETPOINT_OFF_PEAKS* (pour le cas 2). 

In [15]:
from optimiser_engine import StandardWHType 
standard = StandardWHType.SETPOINT  # Cas standard. 
hc_seulement = StandardWHType.SETPOINT_OFF_PEAK  #Heures creuses seulement. 

A part le mode du chauffe-eau, il faut également fournir la température de consigne.
 
La fonction **trajectory_of_client_standard** prend en argument : 
- Un client. 
- Un tableau de production solaire (même conditions que l'optimisé). 
- Température initiale. 
- Instant initial. 
- Un mode du chauffe eau (StandardWHType). 
- Une température de consigne. 

Voici un exemple de code qui appelle cette fonction : 

In [16]:
trajectoire_standard = service.trajectory_of_client_standard(client=client, 
                                                             start_datetime=instant_initial, 
                                                             initial_temperature=temperature_initiale, 
                                                             production_df=df_productions, 
                                                             mode_WH = StandardWHType.SETPOINT, 
                                                             setpoint_temperature=60)

/Users/anaselb/Dev/libs_optimasol/optimiser_engine_v2.0/src/optimiser_engine/engine/service.py:384: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  target_index = pd.date_range(
/Users/anaselb/Dev/libs_optimasol/optimiser_engine_v2.0/src/optimiser_engine/engine/models/trajectory.py:230: UpdateRequired: La partie décisions (x) du vecteur objectif X a été modifiée avec succès. Toutefois, il faut lancer la fonction update_X() afin de mettre à jour les autres éléments de X.Ceux-ci sont vides en ce moment (np.nan)
  warnings.warn("La partie décisions (x) du vecteur objectif X a été modifiée avec succès. " \


# Génération de la trajectoire Router Only :
Toujours pour pouvoir comparer le scénario optimisé avec le scénario où il y a un routeur, il y a une fonction que présente le service **trajectory_of_client_router**. 
Cette fonction génère une trajectoire que le routeur applique sans optimisation. 

On distingue entre deux cas d'utilisation : 
- Mode Autoconsommation seul : Le routeur est chauffé jusqu'à la température consigne uniquement via le solaire. 
- Mode confort : Le routeur peut envoyer l'électricité des heures creuses pour combler une perte de température. 

Pour gérer cette distinction, il faut instancier une classe **RouterMode**, en voilà un exemple :

In [17]:
from optimiser_engine import RouterMode 
autoconsommation_seule = RouterMode.SELF_CONSUMPTION_ONLY 
heures_creuses_aussi = RouterMode.COMFORT 

La fonction **trajectory_of_client_router** prend ces paramètres : 
- Un client (Client).
- Un tableau de solaire (même conditions que l'optimisé).
- Un instant initial (datetime).
- Une température initiale. 
- Une température de consigne. 
- Un mode de routeur. 

Voici un exemple de calcul d'une trajectoire routeur : 


In [18]:
trajectoire_routeur = service.trajectory_of_client_router(client=client, 
                                                          start_datetime=instant_initial, 
                                                          initial_temperature=temperature_initiale, 
                                                          production_df=df_productions, 
                                                          router_mode=RouterMode.SELF_CONSUMPTION_ONLY, 
                                                          setpoint_temperature=60) 

/Users/anaselb/Dev/libs_optimasol/optimiser_engine_v2.0/src/optimiser_engine/engine/service.py:384: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  target_index = pd.date_range(
